<a href="https://colab.research.google.com/github/vardansaini/Custom-T5/blob/main/Huggingface_PretrainedT5_compound_word_embeddings_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is working code and was built before building a Custom pytorch model, the issue is that prediction works but sometimes gives a blank output or an ouput that does not match.

In [1]:
!pip install transformers
!pip install datasets
!pip install SentencePiece
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 3.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from transformers import EncoderDecoderModel, BertGenerationTokenizer, BertTokenizer, BertConfig, EncoderDecoderConfig, EncoderDecoderModel, BertTokenizerFast, Seq2SeqTrainingArguments, Seq2SeqTrainer, BertGenerationEncoder, BertGenerationDecoder, T5Tokenizer, T5ForConditionalGeneration
# import torch
from scipy import spatial

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset
dict_words = pd.read_csv('/content/drive/MyDrive/unigram_freq.csv')
dict_words = dict_words.drop('count', axis='columns')


clean = dict_words.dropna()
words = clean['word'].to_list()
print(len(words))

data2 = Dataset.from_list([{"x": x, "y": x} for x in words])

check = data2['x']
print(len(check[:100]))

In [ ]:
encoder_max_length=16
decoder_max_length=16



def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["x"], add_special_tokens=False, return_tensors="pt", padding="max_length", truncation=True, max_length=encoder_max_length)

  outputs = tokenizer(batch["y"],add_special_tokens=False, return_tensors="pt", padding="max_length", truncation=True, max_length=decoder_max_length)


  newbatch = {}
  newbatch["input_ids"] = inputs.input_ids
  newbatch["attention_mask"] = inputs.attention_mask
  # newbatch["decoder_input_ids"] = outputs.input_ids
  newbatch["decoder_attention_mask"] = outputs.attention_mask
  newbatch["labels"] = outputs.input_ids.clone()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`.
  # We have to make sure that the PAD token is ignored
  # batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return newbatch

filldata = data2
# print(filldata['x'])


filldata = filldata.map(process_data_to_model_inputs, batched=True, batch_size=100)
filldata.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_attention_mask", "labels"],
)

print(filldata)



In [ ]:
ladec = pd.read_csv('/content/drive/MyDrive/ladec.csv')
correct = ladec[ladec["correctParse"] == 'yes']
print(len(correct["c2"]))
# correct = correct[correct["c2"].str[-1] != "s"]
correct = correct.sample(frac=1, random_state=1)
c1, c2, compounds = correct['c1'].to_list(), correct['c2'].to_list(), correct['stim'].to_list()
print(c1)

In [ ]:
sentences = [x + " " + y for x, y in zip(c1, c2)]
print(type(sentences))
print(len(sentences))

X_labels = [ 'X_%02d' % x for x in range(16) ]
Y_labels = [ 'Y_%02d' % y for y in range(16) ]

inputX = pd.DataFrame( columns = ['input_samplesX', *X_labels])
inputY = pd.DataFrame( columns = ['input_samplesY', *Y_labels])
inputX
df1 = pd.DataFrame()

In [ ]:

data = Dataset.from_list([{"x": x, "y": y} for x, y in zip(sentences, compounds)])

encoder_max_length=16
decoder_max_length=16



def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  # print(len(batch["x"]))
  # print(batch["x"])
  # print(batch["y"])
  inputs = tokenizer(batch["x"], add_special_tokens=False, return_tensors="pt", padding="max_length", truncation=True, max_length=encoder_max_length)
  # df = pd.DataFrame(batch["x"])
  # print(len(df))
  # inputX[['input_samplesX']] = df
  # df = pd.DataFrame(inputs.input_ids)
  # print(len(df))
  # print(df[0])
  # count = 0
  # for column in inputX.columns[1:]:
  #   print(len(df[count]))
  #   inputX[column] = df[count]
  #   count += 1

  # print(inputX)
  # print(df1)
  # pd.concat([df1, inputX], ignore_index = True, axis = 1)
  # print(df1)

  # df = pd.DataFrame(inputs.input_ids)
  # inputX[['embeddings']] = df
  # print("input size", inputs.input_ids[0])
  # print("input size", len(inputs.input_ids.size()))
  outputs = tokenizer(batch["y"],add_special_tokens=False, return_tensors="pt", padding="max_length", truncation=True, max_length=decoder_max_length)
  # df = pd.DataFrame(batch["y"])
  # print(len(df))
  # inputY[['input_samplesY']] = df
  # df = pd.DataFrame(outputs.input_ids)
  # print(len(df))
  # print(df[0])
  # count = 0
  # for column in inputY.columns[1:]:
  #   print(len(df[count]))
  #   inputY[column] = df[count]
  #   count += 1

  # print(inputX)
  # print("output size", len(outputs.input_ids.size()))

  newbatch = {}
  newbatch["input_ids"] = inputs.input_ids
  newbatch["attention_mask"] = inputs.attention_mask
  # newbatch["decoder_input_ids"] = outputs.input_ids
  newbatch["decoder_attention_mask"] = outputs.attention_mask
  newbatch["labels"] = outputs.input_ids.clone()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`.
  # We have to make sure that the PAD token is ignored
  # batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return newbatch

traindata = data.select(range(8000))


traindata = traindata.map(process_data_to_model_inputs, batched=True, batch_size=100, remove_columns=["x", "y"])
traindata.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_attention_mask", "labels"],
)

vdata = data.select(range(8000, 8100))
vdata = vdata.map(process_data_to_model_inputs, batched=True, batch_size=100, remove_columns=["x", "y"])
vdata.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_attention_mask", "labels"],
)

# print("I AM HERE AND ABOUT TO SAVE")
# print(df1)
# inputX.to_csv('/content/drive/MyDrive/transformers_inputX.csv')
# inputY.to_csv('/content/drive/MyDrive/transformers_inputY.csv')
# data = data.remove_columns(["x", "y"])

Next three block are just for checking data.

In [ ]:
# print(data['x'])
# print(data['y'])


# count = 0
# for i in range(0, len(data['x'])-1):
#   combine = data['x'][i].replace(" ", "")
#   # print(combine + " =? " + data['y'][i])
#   if combine !=  data['y'][i]:
#     count += 1
# print(count)

In [ ]:
# print(len(traindata))
# print(len(traindata['input_ids'][0]))
# print(traindata['input_ids'][0])
# print(len(traindata['labels'][0]))
# print(traindata['labels'][0])

# len_input_ids = len(traindata['input_ids'][0])
# len_labels = len(traindata['labels'][0])

# count_input = 0
# count_labels = 0
# for i in range(1, len(traindata)-1):
#   if len_input_ids != len(traindata['input_ids'][i]):
#     count_input += 1
#     print(len(traindata['input_ids'][i]))
#     print(traindata['input_ids'][i])
#   if len_labels != len(traindata['labels'][i]):
#     count_labels += 1
#     print(len(traindata['labels'][i]))
#     print(traindata['labels'][i])
# print("count_input ", count_input)
# print("count_labels", count_labels)

# # import pandas as pd
# # test = pd.DataFrame( columns = ['input'])
# # df = pd.DataFrame(traindata['labels'])
# # test[['input']] = df
# # # print(traindata['pred'])
# # test.to_csv('/content/drive/MyDrive/transformers_traindata.csv')

In [ ]:
# print(len(vdata))
# print(len(vdata['input_ids']))
# print(vdata['input_ids'])
# print(len(vdata['input_ids'][0]))
# print(vdata['input_ids'][0])
# print(len(vdata['labels'][0]))
# print(vdata['labels'][0])

# len_input_ids = len(vdata['input_ids'][0])
# len_labels = len(vdata['labels'][0])

# count_input = 0
# count_labels = 0
# for i in range(1, len(vdata)-1):
#   if len_input_ids != len(vdata['input_ids'][i]):
#     count_input += 1
#     print(len(vdata['input_ids'][i]))
#     print(vdata['input_ids'][i])
#   if len_labels != len(vdata['labels'][i]):
#     count_labels += 1
#     print(len(vdata['labels'][i]))
#     print(vdata['labels'][i])
# print("count_input ", count_input)
# print("count_labels", count_labels)


In [ ]:
# model.config.vocab_size = model.config.decoder.vocab_size
# model.config.decoder_start_token_id = tokenizer.cls_token_id
# model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
# !pip install transformers[torch]

In [ ]:
# !pip install accelerate -U

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from transformers.optimization import Adafactor, AdafactorSchedule
# from torch import nn
from sklearn.metrics import mean_squared_error

# !pip install transformers[torch]

optimizer = Adafactor(model.parameters(),
    lr=1e-5,
    eps=(1e-40, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.2,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False,)
lr_scheduler = AdafactorSchedule(optimizer)

training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    evaluation_strategy="steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs = 5,
    seed=0,
    load_best_model_at_end = True,
)

early_stop = EarlyStoppingCallback(3, 0.1)

# class NewTrainer(Seq2SeqTrainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.get("labels")
#         outputs = model(**inputs)
#         logits = outputs.get('logits')
#         print(logits[0][0][0])
#         # loss_fct = nn.MSELoss()
#         # print(self.model.config.num_labels)
#         # print(logits.view(-1, self.model.config.num_labels))
#         # print(labels.view(-1))
#         # loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
#         loss_fct = nn.MSELoss()
#           # if self.num_labels == 1:
#         loss = loss_fct(logits.squeeze(), labels.squeeze())
#           # else:
#         # loss = loss_fct(logits, labels)
#         return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    print(len(preds[0]), "LENGTH OF PREDS")
    print(preds[0])

    decoded_preds = tokenizer.batch_decode(preds[0], skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    rmse = mean_squared_error(decoded_preds, decoded_labels, squared=False)
    return {"rmse": rmse}

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     rmse = mean_squared_error(labels, predictions, squared=False)
#     return {"rmse": rmse}

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=traindata,
    eval_dataset=vdata,
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model),
    # callbacks=[early_stop],
    # optimizers=(optimizer, lr_scheduler),
    # compute_metrics=compute_metrics
)
# model.to("cuda")
trainer.train()

In [ ]:
# data = data.remove_columns(["y"])

In [ ]:
import datasets
from transformers import BertTokenizer, EncoderDecoderModel
import pandas as pd

input_labels = [ 'input_%02d' % y for y in range(16) ]
output_labels = [ 'output_%02d' % y for y in range(16) ]
# test = pd.DataFrame( columns = ['test_samples', 'output'])
testI = pd.DataFrame( columns = ['test_samples', *input_labels])
testO = pd.DataFrame( columns = ['test_samples', *output_labels])
test = pd.DataFrame( columns = ['test_samples', 'output', *output_labels])
print(testI)

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("./checkpoint-2500")
model.to("cuda")

test_data =  data.select(range(8100, 8200))
print(test_data)

# # only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# print("Printing test data ############### ", test_data['x'])
# df = pd.DataFrame(test_data)
# test[['test_samples']] = df
# print(df)

batch_size = 100  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    # inputs = tokenizer(batch["x"], padding="max_length", truncation=True, max_length=4, return_tensors="pt", add_special_tokens=False)
    # print(batch)
    inputs = tokenizer(batch["x"], padding="max_length", truncation=True, max_length=16, return_tensors="pt", add_special_tokens=False)
    # print("batch[x] ################### ",batch['x'])
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    # outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=16)

    iterations = []
    n_steps = 8
    choices_per_step = 5

    for _ in range(n_steps):
      iteration = dict()
      iteration["Input"] = tokenizer.decode(input_ids[0], skip_special_tokens=True)

      # Generate the output logits using the T5 model
      output = model.generate(input_ids, attention_mask=attention_mask, max_length=16)
      next_token_logits = output[0][-1]
      next_token_probs = tf.nn.softmax(next_token_logits.cpu())

      sorted_ids = tf.argsort(
          next_token_probs, axis=-1, direction='DESCENDING', stable=False, name=None
      )

      for choice_idx in range(choices_per_step):
          token_id = sorted_ids[choice_idx]
          token_prob = next_token_probs[token_id].numpy()
          token_choice = f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
          iteration[f"Choice {choice_idx+1}"] = token_choice

      # Append the predicted next token to input_ids
      input_ids = tf.concat([input_ids, sorted_ids[None, 0, None]], axis=-1)
      iterations.append(iteration)

    df = pd.DataFrame(iterations)
    print(df)

    # all special tokens including will be removed
    # output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # df = pd.DataFrame(batch["x"])
    # testI[['test_samples']] = df
    # testO[['test_samples']] = df
    # test[['test_samples']] = df
    # df = pd.DataFrame(inputs.input_ids)

    # count = 0
    # for column in testI.columns[1:]:
    #   print(len(df[count]))
    #   testI[column] = df[count]
    #   count += 1

    # df = pd.DataFrame(outputs.cpu())

    # count = 0
    # for column in testO.columns[1:]:
    #   print(len(df[count]))
    #   testO[column] = df[count]
    #   count += 1

    # count = 0
    # for column in test.columns[2:]:
    #   print(len(df[count]))
    #   test[column] = df[count]
    #   count += 1


    # # print("Inputs ################### ", inputs)
    # # print("Inputs_ids ###################", input_ids[0])
    # # print("Inputs_ids ###################", len(input_ids[0]))
    # # print("outputs ################### ", outputs[0])
    # # print("outputs ################### ", len(outputs[0]))
    # # print("outputs_str ################### ",output_str[0])


    # # print("attention_mask ################### ",attention_mask)

    # batch["pred"] = output_str

    # return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size)
# df = pd.DataFrame(results['pred'])
# test[['output']] = df
# # print(results['pred'])
# testI.to_csv('/content/drive/MyDrive/transformers_test_input.csv')
# testO.to_csv('/content/drive/MyDrive/transformers_test_output.csv')
# test.to_csv('/content/drive/MyDrive/transformers_test_full.csv')